In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from refuge.config import load_config
from refuge.training import train, get_tokenizer_and_model

In [3]:
cfg = load_config()
cfg

namespace(project=namespace(name='alice'),
          model=namespace(hugging_face_name='databricks/dolly-v2-3b'),
          prompt=namespace(initializer="Write an exerpt of a surreal children's fantasy story set in a subterranean world populated by peculiar anthropomorphic creatures. Go!\n\n"),
          training=namespace(block_size=300,
                             checkpoint_interval=20,
                             eval_interval=5,
                             eval_blocks=8,
                             batch_size=1,
                             base_acc_steps=16,
                             acc_doubling_rate=0,
                             plateau_steps=0),
          optimizer=namespace(lr=0.01,
                              beta1=0.0,
                              decay_rate=-0.8,
                              weight_decay=0.1,
                              scale_parameter=False,
                              relative_step=False),
          scheduler=namespace(num_warmup_steps=0

In [ ]:
tokenizer, model = get_tokenizer_and_model(cfg)

In [ ]:
# parameters = model.gpt_neox.parameters()

In [ ]:
# next(parameters).shape

In [ ]:
# 2560 * 4

In [ ]:
nearest_tokens_for_soft_prompt = model.translated_soft_prompt()
tokenizer.decode(nearest_tokens_for_soft_prompt)

In [ ]:
eos_token_id = tokenizer.encode("### End")[0]

In [ ]:
# prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

# ### Instruction:
# {model.soft_prompt}

# ### Response:"""

# call = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

# output = model.generate(
#     input_ids=call,
#     pad_token_id=tokenizer.pad_token_id,
#     max_new_tokens=1024,
#     top_p=0.92,
#     do_sample=True,
#     eos_token_id=eos_token_id,
# )

# print(tokenizer.decode(output[0]))

In [ ]:
import torch
import torch.nn as nn
from typing import cast

In [ ]:
example_tokens = torch.IntTensor([0, 1, 2]).to(model.device)

In [ ]:
example_embeddings = model.convert_tokens_to_embeddings(example_tokens)

In [ ]:
def max_cosine_similarity(model, embeddings):
    embeddings_transposed = embeddings.T
    normalized_embedding_transposed = torch.linalg.norm(
        embeddings_transposed, dim=0, keepdim=True
    )

    embedding_layer = cast(nn.Embedding, model.gpt_neox.embed_in)
    normalized_weights = torch.linalg.norm(
        embedding_layer.weight, dim=1, keepdim=True
    )

    cosine_similarities = (
        (embedding_layer.weight @ embeddings_transposed)
        / (normalized_weights @ normalized_embedding_transposed)
    ).T
    
    return torch.max(cosine_similarities, dim=1)


max_cosine_similarities = max_cosine_similarity(model, example_embeddings)

In [ ]:
max_soft_prompt_sim = max_cosine_similarity(model, model.soft_prompt_embeddings)
max_soft_prompt_sim.values

In [ ]:
max_token_affinity = max_soft_prompt_sim.values - 1e-5
target = torch.ones_like(max_token_affinity)

In [ ]:
nn.functional.binary_cross_entropy(max_token_affinity, target)

In [ ]:
train(cfg, tokenizer, model)